In [1]:
import os
import shutil

import csv
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
# import tensorflow_text as text
import keras

from keras import Input
from keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import (
    GRU,
    Conv1D,
    Dense,
    Embedding,
    Flatten,
    Lambda,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

print(tf.__version__)

2023-02-16 15:57:36.876052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 15:57:37.712301: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-02-16 15:57:37.712405: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

2.11.0


In [2]:
url = 'https://drive.google.com/file/d/16iYEWb1xCiuR5_TM47RVZyzLFwD7gZnT/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
#shuffle data
data = df.sample(frac=1)

In [3]:
MAX_LEN = 22
MAX_TOKENS = 100000

preprocessing_layer = TextVectorization(
    output_sequence_length=MAX_LEN, 
    max_tokens=MAX_TOKENS
)
preprocessing_layer.adapt(data.abstract[:10000])


2023-02-16 15:57:44.095726: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 15:57:44.107419: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 15:57:44.109195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 15:57:44.111849: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [4]:
preprocessing_layer(data.abstract[:10000])

<tf.Tensor: shape=(10000, 22), dtype=int64, numpy=
array([[   3, 1162,   88, ...,   88, 1231,   25],
       [   2,   40,   26, ...,    6,  188, 2929],
       [ 230,    6,   18, ...,  168,   92, 2425],
       ...,
       [   9, 1380,   86, ...,  376, 3623,  336],
       [   2,   40,   26, ...,    8,   19,    3],
       [   3,  463,  150, ...,   96, 6061, 1620]])>

In [5]:
#one hot encode target
# classes = [[0,1,2,3,4,5,6,7][source] for source in dataSort.ipc_section]
classes = [{"A":0,"B":1,"C":2,"D":3,"E":4,"F":5,"G":6,"H":7}[source] for source in data.ipc_section]
target = to_categorical(classes)
#print target
# target[2002]
len(target)

100000

In [7]:
BATCHSIZE = 50

inputs = Input(shape=(1,),dtype=tf.string,name="InputString")
x = preprocessing_layer(inputs)

# x = Dense(256,activation="relu")(inputs)
# x = Dense(256,activation="relu",kernel_regularizer=tf.keras.regularizers.L2(0.02))(inputs)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = Dense(8,activation="softmax")(inputs)

model = keras.Model(inputs=inputs,outputs=outputs,name="modeltest")
model.summary()
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["Accuracy"])

Model: "modeltest"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputString (InputLayer)    [(None, 512)]             0         
                                                                 
 dense (Dense)               (None, 8)                 4104      
                                                                 
Total params: 4,104
Trainable params: 4,104
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelhistory = model.fit(
    embed(data.abstract[:20000]),
    target[:20000], 
    epochs=50,
    batch_size=BATCHSIZE, 
    validation_split=0.2
    # validation_data=(data.text[6000:],target[6000:]),
    # callbacks=[EarlyStopping(patience=10)]
)

In [ ]:
pd.DataFrame(modelhistory.history)[["loss","val_loss"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")
pd.DataFrame(modelhistory.history)[["Accuracy","val_Accuracy"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")